In [1]:
from docx import Document
from docx.shared import Inches
import pandas as pd
import datetime
import sys
import os
import comtypes.client

In [2]:
# database of all person to whome we have to sent the email
data = pd.read_csv('invitation_data.csv')

**FORMAT OF REPLACING DATA**<br>
replaced_word = [salutation, firstname,secondname,Designation, department, institution,Date,sir_madam,Content]

In [3]:
def sent_time():
    '''
    This function will give the time at which mail is sent
    '''
    x = datetime.datetime.now()
    date = x.strftime("%B %d,%Y")
    return date

In [4]:
def content():
    ''''
    This is content function in which we will give the content of the invitation
    '''
    content = 'Lorem ipsum dolor sit amet, ferri consul ius ut. Vis invidunt salutandi qualisque in, id sed nominati accommodare. Tale errem adipiscing pro te. Has amet iriure gloriatur ea. Probatus intellegebat eu qui.'
    return content

In [5]:
def data_extractor(ID, database):
    '''
    It will extract data from database that will replace that values from the invitation template
    '''
    word_to_be_replaced = ['Salutation', 'first_name','second_name','designation', 'department', 'institution']
    replaced_word = []
    for i in word_to_be_replaced:
        replaced_word.append(database.loc[database.ID == ID, i][ID-1])
    if (database.loc[database.ID == ID, 'gender'][ID-1]) == 'Male':
        replaced_word.insert(7, 'Sir')
    else:
        replaced_word.insert(7, 'Madam')
    return replaced_word

In [6]:
def create_invitation(replaced_word):
    '''
    This will modify invitation template and save it in docx and then convert it to pdf and then delete the docx file.
    '''
    doc = Document('invitation_template.docx') 
    word_to_be_replaced = ['Salutation', 'firstname','secondname','Designation', 'department', 'institution','Date','sir/madam','Content']
    bag_of_word = list(zip(word_to_be_replaced, replaced_word))
    for old_word,new_word in bag_of_word:
        for p in doc.paragraphs:
            inline = p.runs
            for i in range(len(inline)):
                text = inline[i].text
                if old_word in text:
                    text=text.replace(old_word,new_word)
                    inline[i].text = text
    path = os.getcwd()
    abs_path = path+'\\Invitation_Sent\\'
    try:
        os.makedirs(abs_path)
    except:
        pass
    finally:
        doc.save(abs_path+bag_of_word[0][1]+'_'+bag_of_word[1][1]+'_'+bag_of_word[2][1]+'_invitation.docx')
        
    wdFormatPDF = 17
    in_file = os.path.abspath(abs_path+bag_of_word[0][1]+'_'+bag_of_word[1][1]+'_'+bag_of_word[2][1]+'_invitation.docx')
    out_file = os.path.abspath(abs_path+bag_of_word[0][1]+'_'+bag_of_word[1][1]+'_'+bag_of_word[2][1]+'_invitation.pdf')
    word = comtypes.client.CreateObject('Word.Application')
    doc = word.Documents.Open(in_file)
    doc.SaveAs(out_file, FileFormat=wdFormatPDF)
    doc.Close()
    word.Quit()
    os.remove(abs_path+bag_of_word[0][1]+'_'+bag_of_word[1][1]+'_'+bag_of_word[2][1]+'_invitation.docx')

In [7]:
def send_invitation(replaced_word):
    '''
    This function will send the email to all the valid email ids given in the database.
    '''
    # invitation file name --> replaced_word[0]+'_'replaced_word[1]+'_'+replaced_word[2]+'_invitation.pdf'
    filename = (replaced_word[0]+'_'+replaced_word[1]+'_'+replaced_word[2]+'_invitation.pdf')
    print(filename+'-----> FILE SEND')
    #code to send invitation

In [8]:
def __main__():    
    '''
    This is the driving function
    '''
    for id_ in data['ID']:
        if (data.loc[data['ID'] == id_]['invitation_status'] != 'sent')[id_-1]:
            date_= sent_time()
            content_ = content()
            replaced_word = data_extractor(id_, data)
            replaced_word.insert(6, date_)
            replaced_word.insert(8, content_)
            create_invitation(replaced_word)
            send_invitation(replaced_word) #<<======================================================================
            # email sent then update database
            data.loc[data.ID == id_, 'invitation_status'] = 'sent'
            data.to_csv('invitation_data.csv', index=False)

In [9]:
__main__()